<a href="https://colab.research.google.com/github/lanehale/airline-chatbot/blob/main/pytorch07_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# See if tensorboard exists, if not, install it
try:
  from torch.utils.tensorboard import SummaryWriter
  print("SummaryWriter already installed.")
except:
  print("Installing SummaryWriter...")
  !pip install -q tensorboard
  from torch.utils.tensorboard import SummaryWriter
  print("Done installing SummaryWriter.")

SummaryWriter already installed.


In [ ]:
"""
Create the going_modular folder and move in its scripts.
"""
import os

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
  from going_modular import data_setup, engine
  print("going_modular scripts already downloaded.")
except:
  # Get the going_modular scripts
  print("Downloading going_modular scripts...")
  !git clone https://github.com/lanehale/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !rm -rf pytorch-deep-learning
  print("going_modular downloaded.")
  from going_modular import data_setup, engine

print(">!ls going_modular")
!ls going_modular

Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 408 (delta 121), reused 48 (delta 48), pack-reused 259 (from 2)
Receiving objects: 100% (408/408), 5.83 MiB | 17.47 MiB/s, done.
Resolving deltas: 100% (238/238), done.
going_modular downloaded.
>!ls going_modular
data_setup.py	  get_custom_data.py  pretrained_confmat.py  utils.py
download_data.py  get_data.py	      pretrained_writer.py
engine.py	  model_builder.py    __pycache__
get_any_data.py   predict.py	      train.py


In [ ]:
import torch
import torchvision

from pathlib import Path
from going_modular import download_data

# Download 10 percent and 20 percent training data
data_10_percent_path = download_data.from_path(from_path="pizza_steak_sushi.zip", image_dir="pizza_steak_sushi")
data_20_percent_path = download_data.from_path(from_path="pizza_steak_sushi_20_percent.zip", image_dir="pizza_steak_sushi_20_percent")

# Set up training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths
test_dir_10 = data_10_percent_path / "test"
test_dir_20 = data_20_percent_path / "test"

test_image_path_list_10 = list(Path(test_dir_10).glob("*/*.jpg"))  # this is only used for predictions
test_image_path_list_20 = list(Path(test_dir_20).glob("*/*.jpg"))  # this is only used for predictions

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza_steak_sushi data...
>!ls data/pizza_steak_sushi
test
train

Did not find data/pizza_steak_sushi_20_percent directory, creating one...
Unzipping pizza_steak_sushi_20_percent data...
>!ls data/pizza_steak_sushi_20_percent
test
train



## 1. Pick a larger model from torchvision.models to add to the experiments

In [ ]:
weights_b3 = torchvision.models.EfficientNet_B3_Weights.DEFAULT  # .DEFAULT = best available weights from pretraining on ImageNet
weights_b4 = torchvision.models.EfficientNet_B4_Weights.DEFAULT

"""
model_b3 = torchvision.models.efficientnet_b3(weights=weights_b3)
model_b4 = torchvision.models.efficientnet_b4(weights=weights_b4)

print(f"effnetb3 - {model_b3.classifier}")
print(f"effnetb4 - {model_b4.classifier}")

effnetb3 - Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)
effnetb4 - Sequential(
  (0): Dropout(p=0.4, inplace=True)
  (1): Linear(in_features=1792, out_features=1000, bias=True)
)
"""

# Create models list (need to create a new model for each experiment)
model_parameters = {"effnetb3": {"weights": weights_b3, "in_features": 1536, "dropout": 0.3},
                    "effnetb4": {"weights": weights_b4, "in_features": 1792, "dropout": 0.4},
                    }

# Create datasets list
datasets = [["data_10_percent", train_dir_10_percent, test_dir_10, test_image_path_list_10],
            ["data_20_percent", train_dir_20_percent, test_dir_20, test_image_path_list_20],
            ]

# Create epochs list
num_epochs = [10]

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
experiment_number = 12

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in datasets:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      if model_name == "effnetb3":
        model_to_train = torchvision.models.efficientnet_b3(weights=weights_b3).to(device)
      elif model_name == "effnetb4":
        model_to_train = torchvision.models.efficientnet_b4(weights=weights_b4).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 134MB/s]


[INFO] Created SummaryWriter, saving to: runs/2025-05-31/data_10_percent/effnetb3/10_epochs...
Training with model 12_effnetb3_data_10_percent_10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0569 | train_acc: 0.3945 | test_loss: 1.0180 | test_acc: 0.5900
Epoch: 2 | train_loss: 0.9066 | train_acc: 0.7188 | test_loss: 0.9257 | test_acc: 0.6619
Epoch: 3 | train_loss: 0.8059 | train_acc: 0.8633 | test_loss: 0.7251 | test_acc: 0.9062
Epoch: 4 | train_loss: 0.6817 | train_acc: 0.8281 | test_loss: 0.6604 | test_acc: 0.9062
Epoch: 5 | train_loss: 0.6480 | train_acc: 0.8594 | test_loss: 0.6347 | test_acc: 0.9072
Epoch: 6 | train_loss: 0.6505 | train_acc: 0.7539 | test_loss: 0.5862 | test_acc: 0.9271
Epoch: 7 | train_loss: 0.5856 | train_acc: 0.8203 | test_loss: 0.6391 | test_acc: 0.7955
Epoch: 8 | train_loss: 0.5738 | train_acc: 0.8203 | test_loss: 0.5570 | test_acc: 0.8674
Epoch: 9 | train_loss: 0.4734 | train_acc: 0.9453 | test_loss: 0.5427 | test_acc: 0.8873
Epoch: 10 | train_loss: 0.5283 | train_acc: 0.8203 | test_loss: 0.5741 | test_acc: 0.8258
[INFO] Total running time: 40.411 seconds
Predicting with data/pizza_steak_sushi/test image_data...
Max test 

  0%|          | 0/10 [00:00<?, ?it/s]

...

[INFO] Created SummaryWriter, saving to: runs/2025-05-31/data_10_percent/effnetb4/10_epochs...
Training with model 14_effnetb4_data_10_percent_10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0827 | train_acc: 0.4805 | test_loss: 1.0397 | test_acc: 0.6619
Epoch: 2 | train_loss: 1.0254 | train_acc: 0.7031 | test_loss: 0.9968 | test_acc: 0.7443
Epoch: 3 | train_loss: 0.9494 | train_acc: 0.8945 | test_loss: 0.9482 | test_acc: 0.7850
Epoch: 4 | train_loss: 0.9575 | train_acc: 0.7852 | test_loss: 0.9280 | test_acc: 0.7955
Epoch: 5 | train_loss: 0.9162 | train_acc: 0.7773 | test_loss: 0.8802 | test_acc: 0.7547
Epoch: 6 | train_loss: 0.8896 | train_acc: 0.8164 | test_loss: 0.8427 | test_acc: 0.8059
Epoch: 7 | train_loss: 0.7754 | train_acc: 0.9297 | test_loss: 0.8147 | test_acc: 0.8153
Epoch: 8 | train_loss: 0.8715 | train_acc: 0.8008 | test_loss: 0.7663 | test_acc: 0.9072
Epoch: 9 | train_loss: 0.7648 | train_acc: 0.8008 | test_loss: 0.7602 | test_acc: 0.8665
Epoch: 10 | train_loss: 0.8770 | train_acc: 0.8164 | test_loss: 0.7643 | test_acc: 0.8362
[INFO] Total running time: 50.943 seconds
Predicting with data/pizza_steak_sushi/test image_data...
Max test 

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0327 | train_acc: 0.6438 | test_loss: 0.9374 | test_acc: 0.8648
Epoch: 2 | train_loss: 0.8801 | train_acc: 0.8396 | test_loss: 0.7926 | test_acc: 0.9352
Epoch: 3 | train_loss: 0.7747 | train_acc: 0.9083 | test_loss: 0.7149 | test_acc: 0.9506
Epoch: 4 | train_loss: 0.6831 | train_acc: 0.9229 | test_loss: 0.6435 | test_acc: 0.9352
Epoch: 5 | train_loss: 0.6337 | train_acc: 0.9042 | test_loss: 0.5850 | test_acc: 0.9443
Epoch: 6 | train_loss: 0.6176 | train_acc: 0.9000 | test_loss: 0.5348 | test_acc: 0.9534
Epoch: 7 | train_loss: 0.5439 | train_acc: 0.8958 | test_loss: 0.4940 | test_acc: 0.9597
Epoch: 8 | train_loss: 0.5113 | train_acc: 0.9167 | test_loss: 0.4628 | test_acc: 0.9381
Epoch: 9 | train_loss: 0.4851 | train_acc: 0.9208 | test_loss: 0.4431 | test_acc: 0.9534
Epoch: 10 | train_loss: 0.4290 | train_acc: 0.9271 | test_loss: 0.4128 | test_acc: 0.9631
[INFO] Total running time: 88.738 seconds
Predicting with data/pizza_steak_sushi_20_percent/test image_data..

In [ ]:
!ls models

12_effnetb3_data_10_percent_10_epochs.pth
13_effnetb3_data_20_percent_10_epochs.pth
14_effnetb4_data_10_percent_10_epochs.pth
15_effnetb4_data_20_percent_10_epochs.pth


In [ ]:
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 37)

12_effnetb3_data_10_percent_10_epochs | False predictions: 11 out of 75 , or 14.67% wrong, 85.33% right
13_effnetb3_data_20_percent_10_epochs | False predictions: 5  out of 150, or  3.33% wrong, 96.67% right
14_effnetb4_data_10_percent_10_epochs | False predictions: 10 out of 75 , or 13.33% wrong, 86.67% right
15_effnetb4_data_20_percent_10_epochs | False predictions: 5  out of 150, or  3.33% wrong, 96.67% right


In [ ]:
index = 0
best_model_index = 0
best_model_acc = 0.0

for experiment in completed_experiments:
  results = completed_experiments[experiment][2]
  max_test_acc = max(results['test_acc'])
  index += 1

  if max_test_acc > best_model_acc:
    best_model_acc = max_test_acc
    best_model_index = index

  print(
      f"{completed_experiments[experiment][0] :<46} | "
      f"Max test acc: {max_test_acc:.3f} | "
      f"Min test loss: {min(results['test_loss']):.3f}"
  )

12_effnetb3_data_10_percent_10_epochs          | Max test acc: 0.927 | Min test loss: 0.543
13_effnetb3_data_20_percent_10_epochs          | Max test acc: 0.966 | Min test loss: 0.264
14_effnetb4_data_10_percent_10_epochs          | Max test acc: 0.907 | Min test loss: 0.760
15_effnetb4_data_20_percent_10_epochs          | Max test acc: 0.963 | Min test loss: 0.413


In [ ]:
# Save the models locally to my machine
from google.colab import files

#skip = True
skip = False

if skip:
  for experiment in completed_experiments:
    model_name = completed_experiments[experiment][0]
    model_path = "models/" + model_name + ".pth"  # gets model from here and
    files.download(model_path)                    # saves it to my Downloads folder
  print("Models downloaded.")
else:
  print("Skipping download of models.")

Skipping download of models.


## 2. Introduce data augmentation to the list of experiments

In [ ]:
import torch
import torchvision

from pathlib import Path

weights_b2 = torchvision.models.EfficientNet_B2_Weights.DEFAULT

# Create models list (need to create a new model for each experiment)
models = ["effnetb2_auto",
          "effnetb2_aug_wide_auto",
          "effnetb2_aug_many_auto",
          "effnetb2_norm",
          "effnetb2_aug_wide_norm",
          "effnetb2_aug_many_norm"]

model_parameters = {"weights": weights_b2, "in_features": 1408, "dropout": 0.3}

# Create datasets list
datasets = [["data_20_percent", train_dir_20_percent, test_dir_20, test_image_path_list_20],
            ]

# Create epochs list
num_epochs = [10]

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Keep track of experiment numbers
experiment_number = 16

completed_experiments = {}

for model_name in models:
  weights = model_parameters["weights"]
  in_features = model_parameters["in_features"]
  dropout = model_parameters["dropout"]

  for data in datasets:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      if model_name == "effnetb2_auto":
        transform = None  # will default to auto transforms in run_model_writer
      elif model_name == "effnetb2_aug_wide_auto":
        transform = transforms.Compose([
            transforms.TrivialAugmentWide(),
            weights.transforms()
        ])
      elif model_name == "effnetb2_aug_many_auto":
        transform = torchvision.transforms.Compose([
            torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.RandomVerticalFlip(p=0.5),
            torchvision.transforms.RandomRotation(degrees=10),
            torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            weights.transforms()
        ])
      elif model_name == "effnetb2_norm":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalize
        ])
      elif model_name == "effnetb2_aug_wide_norm":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.TrivialAugmentWide(),
            transforms.ToTensor(),
            normalize
        ])
      elif model_name == "effnetb2_aug_many_norm":
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((224, 224)),
            torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.RandomVerticalFlip(p=0.5),
            torchvision.transforms.RandomRotation(degrees=10),
            torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            torchvision.transforms.ToTensor(),
            normalize
        ])

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str,
          transform=transform
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 174MB/s]


[INFO] Created SummaryWriter, saving to: runs/2025-06-01/data_20_percent/effnetb2_auto/10_epochs...
Training with model 16_effnetb2_auto_data_20_percent_10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9791 | train_acc: 0.5604 | test_loss: 0.7128 | test_acc: 0.9011
Epoch: 2 | train_loss: 0.7206 | train_acc: 0.8063 | test_loss: 0.5769 | test_acc: 0.9443
Epoch: 3 | train_loss: 0.5971 | train_acc: 0.7937 | test_loss: 0.4906 | test_acc: 0.9437
Epoch: 4 | train_loss: 0.5227 | train_acc: 0.8271 | test_loss: 0.4484 | test_acc: 0.9222
Epoch: 5 | train_loss: 0.4196 | train_acc: 0.8917 | test_loss: 0.3825 | test_acc: 0.9375
Epoch: 6 | train_loss: 0.3838 | train_acc: 0.9083 | test_loss: 0.3493 | test_acc: 0.9443
Epoch: 7 | train_loss: 0.3517 | train_acc: 0.9208 | test_loss: 0.3165 | test_acc: 0.9688
Epoch: 8 | train_loss: 0.3706 | train_acc: 0.9062 | test_loss: 0.3061 | test_acc: 0.9534
Epoch: 9 | train_loss: 0.3070 | train_acc: 0.9396 | test_loss: 0.2988 | test_acc: 0.9597
Epoch: 10 | train_loss: 0.3613 | train_acc: 0.8958 | test_loss: 0.2692 | test_acc: 0.9750
[INFO] Total running time: 56.880 seconds
Predicting with data/pizza_steak_sushi_20_percent/test image_data..

  0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch: 1 | train_loss: 0.9896 | train_acc: 0.5208 | test_loss: 0.7516 | test_acc: 0.8790
Epoch: 2 | train_loss: 0.7384 | train_acc: 0.7646 | test_loss: 0.6229 | test_acc: 0.8943
Epoch: 3 | train_loss: 0.6080 | train_acc: 0.8187 | test_loss: 0.5488 | test_acc: 0.8943
Epoch: 4 | train_loss: 0.5470 | train_acc: 0.8479 | test_loss: 0.5093 | test_acc: 0.8727
Epoch: 5 | train_loss: 0.4403 | train_acc: 0.8708 | test_loss: 0.4377 | test_acc: 0.9034
Epoch: 6 | train_loss: 0.3917 | train_acc: 0.9062 | test_loss: 0.4187 | test_acc: 0.8943
Epoch: 7 | train_loss: 0.3688 | train_acc: 0.9104 | test_loss: 0.3909 | test_acc: 0.9097
Epoch: 8 | train_loss: 0.3850 | train_acc: 0.8938 | test_loss: 0.3798 | test_acc: 0.9068
Epoch: 9 | train_loss: 0.3273 | train_acc: 0.9250 | test_loss: 0.3757 | test_acc: 0.9006
Epoch: 10 | train_loss: 0.3733 | train_acc: 0.8958 | test_loss: 0.3368 | test_acc: 0.9222
[INFO] Total running time: 32.469 seconds
Predicting with data/pizza_steak_sushi_20_percent/test image_data..

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9958 | train_acc: 0.5208 | test_loss: 0.7775 | test_acc: 0.8477
Epoch: 2 | train_loss: 0.7773 | train_acc: 0.7354 | test_loss: 0.6740 | test_acc: 0.8699
Epoch: 3 | train_loss: 0.6779 | train_acc: 0.7729 | test_loss: 0.5715 | test_acc: 0.8909
Epoch: 4 | train_loss: 0.5784 | train_acc: 0.8167 | test_loss: 0.5730 | test_acc: 0.8415
Epoch: 5 | train_loss: 0.5053 | train_acc: 0.8333 | test_loss: 0.5227 | test_acc: 0.8727
Epoch: 6 | train_loss: 0.4435 | train_acc: 0.9021 | test_loss: 0.4910 | test_acc: 0.8483
Epoch: 7 | train_loss: 0.4593 | train_acc: 0.8604 | test_loss: 0.4348 | test_acc: 0.8761
Epoch: 8 | train_loss: 0.4478 | train_acc: 0.8562 | test_loss: 0.4289 | test_acc: 0.8886
Epoch: 9 | train_loss: 0.3985 | train_acc: 0.8979 | test_loss: 0.4257 | test_acc: 0.8915
Epoch: 10 | train_loss: 0.4162 | train_acc: 0.8500 | test_loss: 0.3831 | test_acc: 0.8824
[INFO] Total running time: 35.521 seconds
Predicting with data/pizza_steak_sushi_20_percent/test image_data..

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0070 | train_acc: 0.5208 | test_loss: 0.8049 | test_acc: 0.8108
Epoch: 2 | train_loss: 0.8317 | train_acc: 0.6792 | test_loss: 0.6805 | test_acc: 0.9034
Epoch: 3 | train_loss: 0.7166 | train_acc: 0.7625 | test_loss: 0.6082 | test_acc: 0.8847
Epoch: 4 | train_loss: 0.6587 | train_acc: 0.7625 | test_loss: 0.5892 | test_acc: 0.8818
Epoch: 5 | train_loss: 0.5696 | train_acc: 0.8354 | test_loss: 0.5053 | test_acc: 0.8812
Epoch: 6 | train_loss: 0.5123 | train_acc: 0.8562 | test_loss: 0.5019 | test_acc: 0.8602
Epoch: 7 | train_loss: 0.4923 | train_acc: 0.8542 | test_loss: 0.4662 | test_acc: 0.8665
Epoch: 8 | train_loss: 0.5166 | train_acc: 0.8438 | test_loss: 0.4650 | test_acc: 0.8233
Epoch: 9 | train_loss: 0.4594 | train_acc: 0.8708 | test_loss: 0.4766 | test_acc: 0.8631
Epoch: 10 | train_loss: 0.4855 | train_acc: 0.8250 | test_loss: 0.4494 | test_acc: 0.8511
[INFO] Total running time: 59.121 seconds
Predicting with data/pizza_steak_sushi_20_percent/test image_data..

In [ ]:
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 51)

16_effnetb2_auto_data_20_percent_10_epochs          | False predictions: 4  out of 150, or  2.67% wrong, 97.33% right
17_effnetb2_aug_wide_auto_data_20_percent_10_epochs | False predictions: 5  out of 150, or  3.33% wrong, 96.67% right
18_effnetb2_aug_many_auto_data_20_percent_10_epochs | False predictions: 6  out of 150, or  4.00% wrong, 96.00% right
19_effnetb2_norm_data_20_percent_10_epochs          | False predictions: 7  out of 150, or  4.67% wrong, 95.33% right
20_effnetb2_aug_wide_norm_data_20_percent_10_epochs | False predictions: 8  out of 150, or  5.33% wrong, 94.67% right
21_effnetb2_aug_many_norm_data_20_percent_10_epochs | False predictions: 5  out of 150, or  3.33% wrong, 96.67% right


In [ ]:
for experiment in completed_experiments:
  results = completed_experiments[experiment][2]
  print(
      f"{completed_experiments[experiment][0] :<51} | "
      f"Max test acc: {max(results['test_acc']):.3f} | "
      f"Min test loss: {min(results['test_loss']):.3f}"
  )

16_effnetb2_auto_data_20_percent_10_epochs          | Max test acc: 0.975 | Min test loss: 0.269
17_effnetb2_aug_wide_auto_data_20_percent_10_epochs | Max test acc: 0.931 | Min test loss: 0.319
18_effnetb2_aug_many_auto_data_20_percent_10_epochs | Max test acc: 0.938 | Min test loss: 0.385
19_effnetb2_norm_data_20_percent_10_epochs          | Max test acc: 0.922 | Min test loss: 0.337
20_effnetb2_aug_wide_norm_data_20_percent_10_epochs | Max test acc: 0.891 | Min test loss: 0.383
21_effnetb2_aug_many_norm_data_20_percent_10_epochs | Max test acc: 0.903 | Min test loss: 0.449


## 3. Scale up the dataset by using the entire Food101 dataset from torchvision.models

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Set the root directory where the dataset will be downloaded
root_dir = './data'

# Download all of the Food101 data
datasets.Food101(root=root_dir, download=True)

100%|██████████| 5.00G/5.00G [02:57<00:00, 28.2MB/s]


Dataset Food101
    Number of datapoints: 75750
    Root location: ./data
    split=train

In [ ]:
!ls data/food-101/images -1 | wc -l
!ls data/food-101/images/waffles -1 | wc -l

101
1000


In [ ]:
weights_b2 = torchvision.models.EfficientNet_B2_Weights.DEFAULT
auto_transform = weights_b2.transforms()

# Create train and test datasets with transforms
train_dataset_101 = datasets.Food101(root=root_dir, split='train', transform=auto_transform)
test_dataset_101 = datasets.Food101(root=root_dir, split='test', transform=auto_transform)

len(train_dataset_101), len(test_dataset_101)

(75750, 25250)

In [ ]:
"""
If using torchvision.datasets above, the list of all test tensors is already in test_dataset_101.
We could use a different predict_and_store to convert tensors back to images. But usually models
are validated with part of the test data, and predicted on the rest of the test data. Using the
provided Food101 datasets prevents a problem, because the test dataset won't split evenly across
classes.

If using files, this would be all 101,000 images. We'll need to move 25% of each class' images
to test directories to use this method. And in that case we might as well split the test data
into validation and test sets.
"""
# Get a list of all test images for predictions
image_path_list_101 = list(Path("data/food-101/images").glob("*/*.jpg"))

In [ ]:
len(image_path_list_101)

101000

In [ ]:
# Create a function to display each model's file size
from pathlib import Path

def get_model_size(directory_path):
  for root, subdirectories, files in os.walk(directory_path):
    for file in files:
      model_path = os.path.join(root, file)
      model_size = Path(model_path).stat().st_size // (1024 * 1024)
      print(f"Model size of {model_path} trained with 100% data: {model_size} MB")

### Try three classes (pizza, steak, sushi) with 100% data

In [ ]:
import os
import shutil

def copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n):
  cmd = f"mkdir {train_dest_dir}"
  os.popen(cmd).read()
  cmd = f"mkdir {test_dest_dir}"
  os.popen(cmd).read()

  every_n = 1

  for file_name in os.listdir(source_dir):
    if every_n % n == 0:
      shutil.copy(os.path.join(source_dir, file_name), test_dest_dir)
    else:
      shutil.copy(os.path.join(source_dir, file_name), train_dest_dir)
    every_n += 1

  output = os.popen(f"ls {train_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {train_dest_dir}")
  output = os.popen(f"ls {test_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {test_dest_dir}")

In [ ]:
!mkdir data/food-3
!mkdir data/food-3/images
!mkdir data/food-3/images/train
!mkdir data/food-3/images/test

In [ ]:
n = 1000 / 250  # copy every nth item to test folder

source_dir = 'data/food-101/images/pizza'
test_dest_dir = 'data/food-3/images/test/pizza'
train_dest_dir = 'data/food-3/images/train/pizza'

copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n)

source_dir = 'data/food-101/images/steak'
test_dest_dir = 'data/food-3/images/test/steak'
train_dest_dir = 'data/food-3/images/train/steak'

copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n)

source_dir = 'data/food-101/images/sushi'
test_dest_dir = 'data/food-3/images/test/sushi'
train_dest_dir = 'data/food-3/images/train/sushi'

copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n)

750
 images in data/food-3/images/train/pizza
250
 images in data/food-3/images/test/pizza
750
 images in data/food-3/images/train/steak
250
 images in data/food-3/images/test/steak
750
 images in data/food-3/images/train/sushi
250
 images in data/food-3/images/test/sushi


In [ ]:
# Define data directories
train_dir_3 = 'data/food-3/images/train'
test_dir_3 = 'data/food-3/images/test'
test_images_list_3 = list(Path(test_dir_3).glob("*/*.jpg"))

# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_3_dir", train_dir_3, test_dir_3, test_images_list_3],
               ]

# Create epochs list
num_epochs = [1]

experiment_number = 28

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

### end of experiments ###

print()

# Display results
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

print()

# Display the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)

[INFO] Created SummaryWriter, saving to: runs/2025-06-03/data_food_3_dir/effnetb2/1_epochs...
Training with model 28_effnetb2_data_food_3_dir_1_epochs...


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6465 | train_acc: 0.7953 | test_loss: 0.4128 | test_acc: 0.9085
[INFO] Total running time: 800.913 seconds
Predicting with data/food-3/images/test image_data...
Max test acc: 0.908 | Min test loss: 0.413
[INFO] Saving model to: models/28_effnetb2_data_food_3_dir_1_epochs.pth
--------------------------------------------------


28_effnetb2_data_food_3_dir_1_epochs     | False predictions: 69 out of 750, or  9.20% wrong, 90.80% right

Model size of models/24_effnetb2_data_food_3_1_epochs.pth trained with 100% data: 29 MB
Model size of models/22_effnetb2_data_food_1_1_epochs.pth trained with 100% data: 29 MB
Model size of models/27_effnetb2_data_food_1_dir_1_epochs.pth trained with 100% data: 29 MB
Model size of models/28_effnetb2_data_food_3_dir_1_epochs.pth trained with 100% data: 29 MB
Model size of models/23_effnetb2_data_food_1_1_epochs.pth trained with 100% data: 29 MB
CPU times: user 9min 49s, sys: 5min 4s, total: 14min 54s
Wall time: 15min 5s


### Run the full Food101 dataset

In [ ]:
import os
import shutil

def move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, total_items, num_to_move, split=False):
  cmd = f"mkdir {train_dest_dir}"
  os.popen(cmd).read()
  cmd = f"mkdir {test_dest_dir}"
  os.popen(cmd).read()

  n = total_items // num_to_move  # move every nth item to test_dest folder
  every_n = 1
  count = 0

  for file_name in os.listdir(source_dir):
    if every_n % n == 0 and count < num_to_move:
      shutil.move(os.path.join(source_dir, file_name), test_dest_dir)
      count += 1
    elif split:
      shutil.move(os.path.join(source_dir, file_name), train_dest_dir)
    every_n += 1

  output = os.popen(f"ls {train_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {train_dest_dir}")
  output = os.popen(f"ls {test_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {test_dest_dir}")

In [ ]:
!mkdir data/food-101/images/train
!mkdir data/food-101/images/test
!mkdir data/food-101/images/val

### Each class 700 train, 175 validation, 125 test images

In [ ]:
# Separate data into train and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/{class_name}'
  test_dest_dir = f'data/food-101/images/val/{class_name}'
  train_dest_dir = f'data/food-101/images/train/{class_name}'

  # Move 300 to validation
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 1000, 300, True)

700
 images in data/food-101/images/train/apple_pie
300
 images in data/food-101/images/val/apple_pie
700
 images in data/food-101/images/train/baby_back_ribs
300
 images in data/food-101/images/val/baby_back_ribs
700
 images in data/food-101/images/train/baklava
300
 images in data/food-101/images/val/baklava
700
 images in data/food-101/images/train/beef_carpaccio
300
 images in data/food-101/images/val/beef_carpaccio
700
 images in data/food-101/images/train/beef_tartare
300
 images in data/food-101/images/val/beef_tartare
700
 images in data/food-101/images/train/beet_salad
300
 images in data/food-101/images/val/beet_salad
700
 images in data/food-101/images/train/beignets
300
 images in data/food-101/images/val/beignets
700
 images in data/food-101/images/train/bibimbap
300
 images in data/food-101/images/val/bibimbap
700
 images in data/food-101/images/train/bread_pudding
300
 images in data/food-101/images/val/bread_pudding
700
 images in data/food-101/images/train/breakfast_bu

In [ ]:
# Separate test data into validation and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/val/{class_name}'
  test_dest_dir = f'data/food-101/images/test/{class_name}'
  train_dest_dir = f'data/food-101/images/val/{class_name}'

  # Move 125 to test
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 300, 125)

700
 images in data/food-101/images/train/apple_pie
125
 images in data/food-101/images/test/apple_pie
700
 images in data/food-101/images/train/baby_back_ribs
125
 images in data/food-101/images/test/baby_back_ribs
700
 images in data/food-101/images/train/baklava
125
 images in data/food-101/images/test/baklava
700
 images in data/food-101/images/train/beef_carpaccio
125
 images in data/food-101/images/test/beef_carpaccio
700
 images in data/food-101/images/train/beef_tartare
125
 images in data/food-101/images/test/beef_tartare
700
 images in data/food-101/images/train/beet_salad
125
 images in data/food-101/images/test/beet_salad
700
 images in data/food-101/images/train/beignets
125
 images in data/food-101/images/test/beignets
700
 images in data/food-101/images/train/bibimbap
125
 images in data/food-101/images/test/bibimbap
700
 images in data/food-101/images/train/bread_pudding
125
 images in data/food-101/images/test/bread_pudding
700
 images in data/food-101/images/train/bre

In [ ]:
train_data_101 = "data/food-101/images/train"
val_data_101 = "data/food-101/images/val"
test_data_101 = "data/food-101/images/test"
test_data_path_list_101 = list(Path(test_data_101).glob("*/*.jpg"))

!ls {train_data_101} -1 | wc -l
!ls {val_data_101} -1 | wc -l
!ls {test_data_101} -1 | wc -l
len(test_data_path_list_101)

101
101
101


12625

In [ ]:
# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_101", train_data_101, val_data_101, test_data_path_list_101],
              ]

# Create epochs list
num_epochs = [5]

experiment_number = 24

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

### end of experiments ###

print()

# Display results
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

print()

# Display the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)

[INFO] Created SummaryWriter, saving to: runs/2025-06-02/data_food_101/effnetb2/1_epochs...
Training with model 24_effnetb2_data_food_101_1_epochs...


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.4418 | train_acc: 0.4422 | test_loss: 1.8250 | test_acc: 0.5609
[INFO] Total running time: 823.256 seconds
Predicting with data/food-101/images/val image_data...
Max test acc: 0.561 | Min test loss: 1.825
[INFO] Saving model to: models/24_effnetb2_data_food_101_1_epochs.pth
--------------------------------------------------

CPU times: user 9min, sys: 27.8 s, total: 9min 28s
Wall time: 18min 53s


### Each class 600 train, 200 validation, 200 test images

In [ ]:
# Separate data into train and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/{class_name}'
  test_dest_dir = f'data/food-101/images/val/{class_name}'
  train_dest_dir = f'data/food-101/images/train/{class_name}'

  # Move 400 to validation
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 1000, 400, True)

600
 images in data/food-101/images/train/apple_pie
400
 images in data/food-101/images/val/apple_pie
600
 images in data/food-101/images/train/baby_back_ribs
400
 images in data/food-101/images/val/baby_back_ribs
600
 images in data/food-101/images/train/baklava
400
 images in data/food-101/images/val/baklava
600
 images in data/food-101/images/train/beef_carpaccio
400
 images in data/food-101/images/val/beef_carpaccio
600
 images in data/food-101/images/train/beef_tartare
400
 images in data/food-101/images/val/beef_tartare
600
 images in data/food-101/images/train/beet_salad
400
 images in data/food-101/images/val/beet_salad
600
 images in data/food-101/images/train/beignets
400
 images in data/food-101/images/val/beignets
600
 images in data/food-101/images/train/bibimbap
400
 images in data/food-101/images/val/bibimbap
600
 images in data/food-101/images/train/bread_pudding
400
 images in data/food-101/images/val/bread_pudding
600
 images in data/food-101/images/train/breakfast_bu

In [ ]:
# Separate test data into validation and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/val/{class_name}'
  test_dest_dir = f'data/food-101/images/test/{class_name}'
  train_dest_dir = f'data/food-101/images/val/{class_name}'

  # Move 200 to test
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 400, 200)

200
 images in data/food-101/images/val/apple_pie
200
 images in data/food-101/images/test/apple_pie
200
 images in data/food-101/images/val/baby_back_ribs
200
 images in data/food-101/images/test/baby_back_ribs
200
 images in data/food-101/images/val/baklava
200
 images in data/food-101/images/test/baklava
200
 images in data/food-101/images/val/beef_carpaccio
200
 images in data/food-101/images/test/beef_carpaccio
200
 images in data/food-101/images/val/beef_tartare
200
 images in data/food-101/images/test/beef_tartare
200
 images in data/food-101/images/val/beet_salad
200
 images in data/food-101/images/test/beet_salad
200
 images in data/food-101/images/val/beignets
200
 images in data/food-101/images/test/beignets
200
 images in data/food-101/images/val/bibimbap
200
 images in data/food-101/images/test/bibimbap
200
 images in data/food-101/images/val/bread_pudding
200
 images in data/food-101/images/test/bread_pudding
200
 images in data/food-101/images/val/breakfast_burrito
200
 

In [ ]:
train_data_101 = "data/food-101/images/train"
val_data_101 = "data/food-101/images/val"
test_data_101 = "data/food-101/images/test"
test_data_path_list_101 = list(Path(test_data_101).glob("*/*.jpg"))

!ls {train_data_101} -1 | wc -l
!ls {val_data_101} -1 | wc -l
!ls {test_data_101} -1 | wc -l
len(test_data_path_list_101)

101
101
101


20200

In [ ]:
# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_101", train_data_101, val_data_101, test_data_path_list_101],
              ]

# Create epochs list
num_epochs = [3]

experiment_number = 25

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 114MB/s]


[INFO] Created SummaryWriter, saving to: runs/2025-06-03/data_food_101/effnetb2/3_epochs...
Training with model 25_effnetb2_data_food_101_3_epochs...


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.4993 | train_acc: 0.4367 | test_loss: 1.8469 | test_acc: 0.5528
Epoch: 2 | train_loss: 1.8843 | train_acc: 0.5354 | test_loss: 1.7235 | test_acc: 0.5699
Epoch: 3 | train_loss: 1.7777 | train_acc: 0.5559 | test_loss: 1.6734 | test_acc: 0.5792
[INFO] Total running time: 2185.414 seconds
Predicting with data/food-101/images/val image_data...
Max test acc: 0.579 | Min test loss: 1.673
[INFO] Saving model to: models/25_effnetb2_data_food_101_3_epochs.pth
--------------------------------------------------

CPU times: user 19min 24s, sys: 1min 14s, total: 20min 39s
Wall time: 44min 55s


In [ ]:
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

25_effnetb2_data_food_101_3_epochs       | False predictions: 8520 out of 20200, or 42.18% wrong, 57.82% right


In [ ]:
# Check the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)

Model size of models/25_effnetb2_data_food_101_3_epochs.pth trained with 100% data: 30 MB


### Each class 750 train, 200 validation, 50 test images

In [ ]:
# Separate data into train and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/{class_name}'
  test_dest_dir = f'data/food-101/images/val/{class_name}'
  train_dest_dir = f'data/food-101/images/train/{class_name}'

  # Move 250 to validation
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 1000, 250, True)

In [ ]:
# Separate test data into validation and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/val/{class_name}'
  test_dest_dir = f'data/food-101/images/test/{class_name}'
  train_dest_dir = f'data/food-101/images/val/{class_name}'

  # Move 50 to test
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 250, 50)

In [ ]:
train_data_101 = "data/food-101/images/train"
val_data_101 = "data/food-101/images/val"
test_data_101 = "data/food-101/images/test"
test_data_path_list_101 = list(Path(test_data_101).glob("*/*.jpg"))

!ls {train_data_101} -1 | wc -l
!ls {val_data_101} -1 | wc -l
!ls {test_data_101} -1 | wc -l
len(test_data_path_list_101)

In [ ]:
# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_101", train_data_101, val_data_101, test_data_path_list_101],
              ]

# Create epochs list
num_epochs = [5]

experiment_number = 26

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

### end of experiments ###

print()

# Display results
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

print()

# Display the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)